# Arctic Rivers exploratory data analysis

This notebook is for exploring data from the Arctic Rivers Project, sponsored by NSF NNA. The goal is to see what we are working with here, asking questions that are relevant to how this data could be integrated into NCR or similar.

Context: co-developed with Indigenous partners a model chain that involves dynamic downscaling (4 km) of a calibrated RASM/WRF/CSM configuration for a historical 30 yr period, and two mid-century simulations using the delta-method, and four downscaled CESM2 LE members each for historical and mid-century 30-yr periods. Simulations have been routed to estimate streamflow and (calibrated) river temperatures (paper in press) for AK and YK rivers. 

In [23]:
import numpy as np
import pandas as pd
import xarray as xr

from pathlib import Path

# path to data is /beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/
data_dir = Path("/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/")

## Data package

Looks like all files are netCDFs:

In [15]:
fps = list(data_dir.glob("*"))
assert all(fp == fp2 for fp, fp2 in zip(fps, data_dir.glob("*.nc")))
print(f"Number of files: {len(fps)}")

Number of files: 1056


And all files have a 3 part name structure:

In [17]:
assert all([len(fp.name.split("_")) == 3 for fp in fps])
print("Some random files:")

print(fps[0].name)
print(fps[200].name)
print(fps[999].name)

Some random files:
2056_fC2LE2_climate.nc
2019_hC2LE9_WT.nc
2009_historical_climate.nc


Looks like we have `<year>_<model>_<variables>.nc` , with the following possible values:

In [58]:
years, models_, variables_ = zip(*[fp.name.split(".")[0].split("_") for fp in fps])
models = list(set(models_))
variables = list(set(variables_))
print(f"year -- first year: {min(years)}, last year: {max(years)}")
print(f"models: {sorted(set(models))}")
print(f"scenarios: {set(variables)}")

year -- first year: 1990, last year: 2065
models: ['fC2LE2', 'fC2LE4', 'fC2LE7', 'fC2LE9', 'fPGWh', 'fPGWm', 'hC2LE2', 'hC2LE4', 'hC2LE7', 'hC2LE9', 'historical']
scenarios: {'WT', 'Q', 'climate'}


Let's get an idea of sizes:

In [47]:
rows = []
for model in models:
    for var in variables:
        modvar_sizes = []
        for year in range(1990, 2066):
            fp = data_dir.joinpath(f"{year}_{model}_{var}.nc")
            if not fp.exists():
                continue
            else:
                size = fp.stat().st_size / 1e6
                modvar_sizes.append(size)

        rows.append(
            {
                "model": model,
                "var": var,
                "mean_size": f"{np.mean(modvar_sizes).astype(int)} MB",
                "std_size": f"{np.std(modvar_sizes).astype(int)} MB",
            }
        )

df = pd.DataFrame(rows)

Mean sizes for each:

In [48]:
df.pivot(index="var", columns="model", values="mean_size")

model,fC2LE2,fC2LE4,fC2LE7,fC2LE9,fPGWh,fPGWm,hC2LE2,hC2LE4,hC2LE7,hC2LE9,historical
var,,,,,,,,,,,
Q,50 MB,50 MB,50 MB,50 MB,50 MB,50 MB,50 MB,50 MB,50 MB,50 MB,52 MB
WT,594 MB,594 MB,594 MB,594 MB,595 MB,595 MB,594 MB,594 MB,594 MB,594 MB,595 MB
climate,969 MB,969 MB,971 MB,971 MB,966 MB,969 MB,986 MB,990 MB,987 MB,988 MB,981 MB


Standard deviations:

In [49]:
df.pivot(index="var", columns="model", values="std_size")

model,fC2LE2,fC2LE4,fC2LE7,fC2LE9,fPGWh,fPGWm,hC2LE2,hC2LE4,hC2LE7,hC2LE9,historical
var,,,,,,,,,,,
Q,2 MB,2 MB,2 MB,2 MB,2 MB,2 MB,2 MB,2 MB,2 MB,2 MB,2 MB
WT,28 MB,28 MB,28 MB,28 MB,26 MB,26 MB,28 MB,28 MB,28 MB,28 MB,26 MB
climate,46 MB,50 MB,46 MB,48 MB,46 MB,45 MB,47 MB,47 MB,47 MB,48 MB,45 MB


What about represnetation of models / vairable groups in the dataset?

In [63]:
unique_models, counts = np.unique(models_, return_counts=True)
pd.DataFrame({"Model": models, "File count": counts})

,Model,File count
0,hC2LE4,96
1,hC2LE7,96
2,hC2LE9,96
3,fC2LE7,96
4,fC2LE4,96
5,hC2LE2,96
6,fPGWh,96
7,historical,96
8,fC2LE9,96
9,fPGWm,96


In [66]:
unique_groups, counts = np.unique(variables_, return_counts=True)
pd.DataFrame({"Variable group": unique_groups, "File count": counts})

,Variable group,File count
0,Q,352
1,WT,352
2,climate,352


## File structure

From above, it looks like most files are of similar sizes within variable group and across models. So let's have a look at a file from each variable group. 

In [79]:
tmp_fps = sorted([fp for fp in fps if "fPGWh" in fp.name])[:3]
for fp in tmp_fps:
    print("File", fp)
    print(xr.open_dataset(fp), end="\n\n---------------------------------\n\n")

File /beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/2034_fPGWh_Q.nc
<xarray.Dataset> Size: 50MB
Dimensions:          (time: 365, seg: 34346)
Coordinates:
  * time             (time) datetime64[ns] 3kB 2034-01-01T12:00:00 ... 2034-1...
  * seg              (seg) int64 275kB 81009065 81010344 ... 81019848 81019993
Data variables:
    IRFroutedRunoff  (time, seg) float32 50MB ...
Attributes:
    version:  v1.2.1

---------------------------------

File /beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/2034_fPGWh_WT.nc
<xarray.Dataset> Size: 600MB
Dimensions:      (time: 365, no_seg: 2, hru: 34208)
Coordinates:
  * time         (time) datetime64[ns] 3kB 2034-01-01 2034-01-02 ... 2034-12-31
  * no_seg       (no_seg) int32 8B 1 2
  * hru          (hru) float64 274kB 8.1e+07 8.1e+07 8.1e+07 ... 8.1e+07 8.1e+07
Data variables:
    T_stream     (time, no_seg, hru) float64 200MB ...
    T_air        (time, no_seg, hru) float64 200MB ...
    T_headwater  (time, no_seg, hru) float64 200MB ...

--------